In [1]:
site_url = 'https://ontologia.eximia.co'
endpoint = '/wp-json/wp/v2/posts'

# Collect

In [2]:
import requests
from datetime import datetime, timedelta

In [3]:
! pip install pandas --quiet

import pandas as pd

In [4]:
import requests

def get_post_types(site_url):
    # Endpoint para tipos de posts
    endpoint = '/wp-json/wp/v2/types'
    api_url = f'{site_url}{endpoint}'


    response = requests.get(api_url)
    post_types = response.json()

    if response.status_code == 200:
        data = [{'key': key, 'name': value['name']} for key, value in post_types.items()]
        return pd.DataFrame(data)
    else:
        return pd.DataFrame()
    
def get_classes(site_url):
    df = get_post_types(site_url)
    classes = df[
        ~(df['key'].str.startswith('wp_') | 
          df['key'].str.startswith('nav_') | 
          df['key'].str.contains('jet-engine') | 
          df['key'].str.contains('assets') |
          df['key'].str.contains('classes') |
          df['key'].str.contains('page') |
          df['key'].str.contains('post') | 
          df['key'].str.contains('links') | 
          df['key'].str.contains('media') | 
          df['key'].str.contains('attachment') | 
          df['key'].str.contains('visibility_preset') | 
          df['key'].str.endswith('_links') |
          df['key'].str.endswith('-links')
          )
    ]
    return classes

get_classes('https://ontologia.eximia.co')

,key,name
10,iniciativas,Iniciativas Estratégicas
11,consultores,Consultores
12,clientes,Clientes
13,cases,Cases
15,autores,Autores
18,template,Templates
19,palestras,Palestras e Lições
20,howtos,HOW-TOs
21,ofertas,Ofertas
23,modelos,Modelos


In [5]:
import requests
import pandas as pd

def get_users(site_url):
    api_url = f'{site_url}/wp-json/wp/v2/users'
    
    response = requests.get(api_url)
    
    if response.status_code == 200:
        users_data = response.json()
        df = pd.DataFrame(users_data)
        df.loc[df['name'] == 'elemar', 'name'] = 'Elemar Jr'
        return df
    else:
        return pd.DataFrame() 

get_users(site_url)

,id,name,url,description,link,slug,avatar_urls,meta,_links
0,1,admin,http://ontologia.eximia.co,,https://ontologia.eximia.co/author/admin/,admin,{'24': 'https://secure.gravatar.com/avatar/dd7...,"{'generateblocks_onboarding': [], 'rsssl_two_f...",{'self': [{'href': 'https://ontologia.eximia.c...
1,8,Camila DuoCom,,,https://ontologia.eximia.co/author/duo/,duo,{'24': 'https://secure.gravatar.com/avatar/1c9...,"{'generateblocks_onboarding': [], 'rsssl_two_f...",{'self': [{'href': 'https://ontologia.eximia.c...
2,2,Elemar Jr,,,https://ontologia.eximia.co/author/elemar/,elemar,{'24': 'https://secure.gravatar.com/avatar/c60...,"{'generateblocks_onboarding': [], 'rsssl_two_f...",{'self': [{'href': 'https://ontologia.eximia.c...
3,5,Fernando Neiva Paiva,,,https://ontologia.eximia.co/author/fernandonei...,fernandoneivapaiva,{'24': 'https://secure.gravatar.com/avatar/a44...,"{'generateblocks_onboarding': [], 'rsssl_two_f...",{'self': [{'href': 'https://ontologia.eximia.c...
4,7,Franco Alves,,,https://ontologia.eximia.co/author/francoalves/,francoalves,{'24': 'https://secure.gravatar.com/avatar/058...,"{'generateblocks_onboarding': [], 'rsssl_two_f...",{'self': [{'href': 'https://ontologia.eximia.c...
5,6,Maicon Reichert,,,https://ontologia.eximia.co/author/maiconreich...,maiconreichert,{'24': 'https://secure.gravatar.com/avatar/eaa...,"{'generateblocks_onboarding': [], 'rsssl_two_f...",{'self': [{'href': 'https://ontologia.eximia.c...


In [6]:
def get_posts_from(site_url, post_type_slug, max_days_old):
    after = datetime.now() - timedelta(days=max_days_old)
    after_iso = after.strftime('%Y-%m-%dT%H:%M:%S')

    endpoint = f'/wp-json/wp/v2/{post_type_slug}'
    api_url = f'{site_url}{endpoint}?after={after_iso}'

    response = requests.get(api_url)
    posts = response.json()

    if response.status_code == 200 and isinstance(posts, list):
        data = [{'ID': post['id'], 'Title': post['title']['rendered'], 'Modified': post['modified'], 'AuthorId': post.get('author', 1)} for post in posts]
        df = pd.DataFrame(data)
        return df
    else:
        return pd.DataFrame()      

def get_posts_from_last_week(site_url, post_type_slug):
    return get_posts_from(site_url, post_type_slug, 7) 


get_posts_from_last_week(site_url, 'conceitos')

,ID,Title,Modified,AuthorId
0,1964,Consciência Situacional,2024-04-25T01:29:52,2
1,1910,Lei de Conway,2024-04-24T15:39:02,2


In [7]:
get_posts_from_last_week(site_url, 'ofertas')

,ID,Title,Modified,AuthorId
0,2118,Ferramenta Fábrica de IAs,2024-04-26T18:00:09,7
1,1771,Consultoria para Governança e Estruturação de ...,2024-04-24T14:13:16,7


In [8]:
get_posts_from_last_week(site_url, 'consultores')

,ID,Title,Modified,AuthorId
0,2106,Gabriel Luis de Oliveira,2024-04-26T15:00:21,1


In [9]:

def get_entries_from(site_url, max_days_old):
    classes_df = get_classes(site_url)
    
    entries_list = []
    
    for index, row in classes_df.iterrows():
        post_type_slug = row['key']
        posts_df = get_posts_from(site_url, post_type_slug, max_days_old)
        
        posts_df['ClassKey'] = row['key']
        posts_df['ClassName'] = row['name']
        #posts_df['AuthorName'] = posts_df['AuthorId'].map(user_name_map)
    
        entries_list.append(posts_df)
    
    final_df = pd.concat(entries_list, ignore_index=True)
    
    return final_df
    
def get_entries_from_last_week(site_url):
    return get_entries_from(site_url, 7)

entries = get_entries_from(site_url, 7)

In [11]:
entries

,ID,Title,Modified,AuthorId,ClassKey,ClassName
0,2134.0,Nome da iniciativa,2024-04-28T20:59:50,1.0,iniciativas,Iniciativas Estratégicas
1,2106.0,Gabriel Luis de Oliveira,2024-04-26T15:00:21,1.0,consultores,Consultores
2,2104.0,Tramontina,2024-04-26T14:57:09,1.0,clientes,Clientes
3,2128.0,Como ajudamos a EximiaCo a otimizar a redação ...,2024-04-28T20:34:53,2.0,cases,Cases
4,1903.0,Template para Propostas Comerciais,2024-04-26T16:37:30,7.0,template,Templates
5,1803.0,Template Follow-up Executivo,2024-04-24T14:16:37,5.0,template,Templates
6,2118.0,Ferramenta Fábrica de IAs,2024-04-26T18:00:09,7.0,ofertas,Ofertas
7,1771.0,Consultoria para Governança e Estruturação de ...,2024-04-24T14:13:16,7.0,ofertas,Ofertas
8,1912.0,Coeficiente de Centralidade,2024-04-24T15:37:28,2.0,modelos,Modelos
9,1782.0,Notebook para Análise de Arquivos e Autores ma...,2024-04-24T14:07:57,2.0,ferramentas,Ferramentas


In [12]:
users = get_users(site_url)
users

,id,name,url,description,link,slug,avatar_urls,meta,_links
0,1,admin,http://ontologia.eximia.co,,https://ontologia.eximia.co/author/admin/,admin,{'24': 'https://secure.gravatar.com/avatar/dd7...,"{'generateblocks_onboarding': [], 'rsssl_two_f...",{'self': [{'href': 'https://ontologia.eximia.c...
1,8,Camila DuoCom,,,https://ontologia.eximia.co/author/duo/,duo,{'24': 'https://secure.gravatar.com/avatar/1c9...,"{'generateblocks_onboarding': [], 'rsssl_two_f...",{'self': [{'href': 'https://ontologia.eximia.c...
2,2,Elemar Jr,,,https://ontologia.eximia.co/author/elemar/,elemar,{'24': 'https://secure.gravatar.com/avatar/c60...,"{'generateblocks_onboarding': [], 'rsssl_two_f...",{'self': [{'href': 'https://ontologia.eximia.c...
3,5,Fernando Neiva Paiva,,,https://ontologia.eximia.co/author/fernandonei...,fernandoneivapaiva,{'24': 'https://secure.gravatar.com/avatar/a44...,"{'generateblocks_onboarding': [], 'rsssl_two_f...",{'self': [{'href': 'https://ontologia.eximia.c...
4,7,Franco Alves,,,https://ontologia.eximia.co/author/francoalves/,francoalves,{'24': 'https://secure.gravatar.com/avatar/058...,"{'generateblocks_onboarding': [], 'rsssl_two_f...",{'self': [{'href': 'https://ontologia.eximia.c...
5,6,Maicon Reichert,,,https://ontologia.eximia.co/author/maiconreich...,maiconreichert,{'24': 'https://secure.gravatar.com/avatar/eaa...,"{'generateblocks_onboarding': [], 'rsssl_two_f...",{'self': [{'href': 'https://ontologia.eximia.c...


In [13]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           6 non-null      int64 
 1   name         6 non-null      object
 2   url          6 non-null      object
 3   description  6 non-null      object
 4   link         6 non-null      object
 5   slug         6 non-null      object
 6   avatar_urls  6 non-null      object
 7   meta         6 non-null      object
 8   _links       6 non-null      object
dtypes: int64(1), object(8)
memory usage: 564.0+ bytes


In [32]:
def get_comments(site_url, start_date, end_date):
    endpoint = f"{site_url}/wp-json/wp/v2/comments"
    
    comments = []
    
    params = {
        'after': start_date + 'T00:00:00',
        'before': end_date + 'T23:59:59',
        'per_page': 100,  # Número máximo de comentários por página
        'page': 1
    }
    
    while True:
        response = requests.get(endpoint, params=params)
        data = response.json()
        
        if not data:
            break
        
        comments.extend(data)
        
        params['page'] += 1
    
    df = pd.DataFrame(comments)
    return df

comments_df = get_comments(site_url, '2024-04-20', '2023-04-29')
print(comments_df.head())


Empty DataFrame
Columns: []
Index: []


# Organize

In [14]:
user_name_map = get_users(site_url).set_index('id')['name'].to_dict()
user_name_map

{1: 'admin',
 8: 'Camila DuoCom',
 2: 'Elemar Jr',
 5: 'Fernando Neiva Paiva',
 7: 'Franco Alves',
 6: 'Maicon Reichert'}

In [15]:
entries['AuthorName'] = entries['AuthorId'].map(user_name_map)

In [17]:
entries

,ID,Title,Modified,AuthorId,ClassKey,ClassName,AuthorName
0,2134.0,Nome da iniciativa,2024-04-28T20:59:50,1.0,iniciativas,Iniciativas Estratégicas,admin
1,2106.0,Gabriel Luis de Oliveira,2024-04-26T15:00:21,1.0,consultores,Consultores,admin
2,2104.0,Tramontina,2024-04-26T14:57:09,1.0,clientes,Clientes,admin
3,2128.0,Como ajudamos a EximiaCo a otimizar a redação ...,2024-04-28T20:34:53,2.0,cases,Cases,Elemar Jr
4,1903.0,Template para Propostas Comerciais,2024-04-26T16:37:30,7.0,template,Templates,Franco Alves
5,1803.0,Template Follow-up Executivo,2024-04-24T14:16:37,5.0,template,Templates,Fernando Neiva Paiva
6,2118.0,Ferramenta Fábrica de IAs,2024-04-26T18:00:09,7.0,ofertas,Ofertas,Franco Alves
7,1771.0,Consultoria para Governança e Estruturação de ...,2024-04-24T14:13:16,7.0,ofertas,Ofertas,Franco Alves
8,1912.0,Coeficiente de Centralidade,2024-04-24T15:37:28,2.0,modelos,Modelos,Elemar Jr
9,1782.0,Notebook para Análise de Arquivos e Autores ma...,2024-04-24T14:07:57,2.0,ferramentas,Ferramentas,Elemar Jr


In [18]:
entries['AuthorName'].value_counts()

AuthorName
Elemar Jr               5
admin                   3
Franco Alves            3
Fernando Neiva Paiva    1
Name: count, dtype: int64

In [19]:
entries['ClassName'].value_counts()

ClassName
Templates                   2
Ofertas                     2
Conceitos                   2
Iniciativas Estratégicas    1
Consultores                 1
Clientes                    1
Cases                       1
Modelos                     1
Ferramentas                 1
Name: count, dtype: int64

In [20]:
entries_last_month = get_entries_from(site_url, 30) 

In [21]:
entries_last_month['AuthorName'] = entries_last_month['AuthorId'].map(user_name_map)
entries_last_month

,ID,Title,Modified,AuthorId,ClassKey,ClassName,AuthorName
0,2134.0,Nome da iniciativa,2024-04-28T20:59:50,1.0,iniciativas,Iniciativas Estratégicas,admin
1,2106.0,Gabriel Luis de Oliveira,2024-04-26T15:00:21,1.0,consultores,Consultores,admin
2,2104.0,Tramontina,2024-04-26T14:57:09,1.0,clientes,Clientes,admin
3,2128.0,Como ajudamos a EximiaCo a otimizar a redação ...,2024-04-28T20:34:53,2.0,cases,Cases,Elemar Jr
4,1608.0,Elemar Júnior,2024-04-16T17:27:23,1.0,autores,Autores,admin
5,1903.0,Template para Propostas Comerciais,2024-04-26T16:37:30,7.0,template,Templates,Franco Alves
6,1803.0,Template Follow-up Executivo,2024-04-24T14:16:37,5.0,template,Templates,Fernando Neiva Paiva
7,944.0,Template de playbook para Enabling Teams,2024-04-24T14:16:40,5.0,template,Templates,Fernando Neiva Paiva
8,891.0,Template para Apresentações com Recomendações,2024-04-24T14:16:44,2.0,template,Templates,Elemar Jr
9,1348.0,A Inteligência Artificial Além do Hype,2024-04-23T21:01:05,2.0,palestras,Palestras e Lições,Elemar Jr


In [22]:
entries_last_month['ClassName'].value_counts()

ClassName
Palestras e Lições          10
Ofertas                     10
Frameworks                   7
Conceitos                    5
Templates                    4
Modelos                      3
Ferramentas                  3
Iniciativas Estratégicas     1
Consultores                  1
Clientes                     1
Cases                        1
Autores                      1
Metodologias                 1
Princípios                   1
Name: count, dtype: int64

In [23]:
entries_last_month['AuthorName'].value_counts()

AuthorName
Elemar Jr               26
Franco Alves            11
admin                    4
Fernando Neiva Paiva     4
Maicon Reichert          4
Name: count, dtype: int64

In [24]:
entries

,ID,Title,Modified,AuthorId,ClassKey,ClassName,AuthorName
0,2134.0,Nome da iniciativa,2024-04-28T20:59:50,1.0,iniciativas,Iniciativas Estratégicas,admin
1,2106.0,Gabriel Luis de Oliveira,2024-04-26T15:00:21,1.0,consultores,Consultores,admin
2,2104.0,Tramontina,2024-04-26T14:57:09,1.0,clientes,Clientes,admin
3,2128.0,Como ajudamos a EximiaCo a otimizar a redação ...,2024-04-28T20:34:53,2.0,cases,Cases,Elemar Jr
4,1903.0,Template para Propostas Comerciais,2024-04-26T16:37:30,7.0,template,Templates,Franco Alves
5,1803.0,Template Follow-up Executivo,2024-04-24T14:16:37,5.0,template,Templates,Fernando Neiva Paiva
6,2118.0,Ferramenta Fábrica de IAs,2024-04-26T18:00:09,7.0,ofertas,Ofertas,Franco Alves
7,1771.0,Consultoria para Governança e Estruturação de ...,2024-04-24T14:13:16,7.0,ofertas,Ofertas,Franco Alves
8,1912.0,Coeficiente de Centralidade,2024-04-24T15:37:28,2.0,modelos,Modelos,Elemar Jr
9,1782.0,Notebook para Análise de Arquivos e Autores ma...,2024-04-24T14:07:57,2.0,ferramentas,Ferramentas,Elemar Jr


In [33]:
entries[entries['AuthorName']=="Fernando Neiva Paiva"]

,ID,Title,Modified,AuthorId,ClassKey,ClassName,AuthorName
5,1803.0,Template Follow-up Executivo,2024-04-24T14:16:37,5.0,template,Templates,Fernando Neiva Paiva
